In [ ]:
import GCRCatalogs
import matplotlib.pyplot as plt
import pandas as pd
import corner
import numpy as np
import h5py
from astropy.table import Table
import os

## Read skysim v3.1.0 and cosmoDC2 v1.1.4 catalogs with the GCRCatalogs class

In [ ]:
skysim3 = GCRCatalogs.load_catalog('skysim_v3.1.0')
cosmodc2 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In [ ]:
skysim3_allcols = skysim3.list_all_native_quantities()
cosmodc2_allcols = cosmodc2.list_all_native_quantities()

In [ ]:
skysim3_relevantcols = ['LSST_obs_g','LSST_obs_i','LSST_obs_r','LSST_obs_u','LSST_obs_y','LSST_obs_z','LSST_rest_G','LSST_rest_I','LSST_rest_R',
                        'LSST_rest_U' ,'LSST_rest_Y','LSST_rest_Z','lg_met_mean','lg_met_scatter','log_sm','log_ssfr','obs_sfr','obs_sm',
                        'redshift','sfr','HSC_obs_g','HSC_obs_i','HSC_obs_r','HSC_obs_y','HSC_obs_z','HSC_rest_G','HSC_rest_I','HSC_rest_R',
                        'HSC_rest_Y','HSC_rest_Z', 'diffmah_early_index', 'lg_met_scatter', 
                        'diffstar_u_indx_lo', 'diffstar_u_tau_dep', 'diffmah_mah_logtc', 'diffmah_logmp_fit',
                        'lg_met_mean', 'diffstar_u_qs', 'diffstar_u_q_rejuv', 'diffstar_u_lgmcrit',
                        'diffstar_u_qt', 'diffmah_late_index', 'diffstar_u_q_drop', 'diffstar_u_lgy_at_mcrit',
                        'diffstar_u_indx_hi']

In [ ]:
cosmodc2_relevantcols = ['LSST_filters/magnitude:LSST_u:rest','LSST_filters/magnitude:LSST_g:rest','LSST_filters/magnitude:LSST_r:rest',
                         'LSST_filters/magnitude:LSST_i:rest','LSST_filters/magnitude:LSST_z:rest','LSST_filters/magnitude:LSST_y:rest',
                         'LSST_filters/magnitude:LSST_u:observed','LSST_filters/magnitude:LSST_g:observed','LSST_filters/magnitude:LSST_r:observed',
                         'LSST_filters/magnitude:LSST_i:observed','LSST_filters/magnitude:LSST_z:observed','LSST_filters/magnitude:LSST_y:observed',
                         'baseDC2/sfr','baseDC2/redshift','baseDC2/obs_sfr','baseDC2/obs_sm','baseDC2/sm']

In [ ]:
skysim3_quantities = skysim3.get_quantities(skysim3_relevantcols, native_filters=['healpix_pixel == 9816'])
cosmodc2_quantities = cosmodc2.get_quantities(cosmodc2_relevantcols, native_filters=['healpix_pixel == 9816'])

In [ ]:
skysim3_df = pd.DataFrame(skysim3_quantities)
cosmodc2_df = pd.DataFrame(cosmodc2_quantities)

## Select a random subset of 100k objects per catalog to facilitate plotting

In [ ]:
skysim3_df_100k = skysim3_df.sample(n=100000)
cosmodc2_df_100k = cosmodc2_df.sample(n=100000)

## Compare physical and photometric properties distribution between skysim v3.1.0 and cosmoDC2 v1.1.4

#### Redshift distribution comparison

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25
plt.hist(cosmodc2_df_100k['baseDC2/redshift'], bins=50, lw=2, ls='solid', color='black', histtype='step',density=True, label='cosmoDC2 v1.1.4')
plt.hist(skysim3_df_100k['redshift'], bins=50, lw=2, ls='dashed', color='red', histtype='step',density=True, label='skysim v3.1.0')
plt.legend(loc='upper right', fontsize=20)
plt.xlim(0,3.5)
plt.xlabel(r'redshift', fontsize=20)
plt.ylabel(r'', fontsize=20)
plt.show()

#### Stellar mass - Star formation rate plane comparison using values quoted in the catalogs. For skysim, these values refer to the stellar mass and star-formation rate evaluated at the last age bin of the galaxy SFH, i.e. 13.7 Gyr.

In [ ]:
w = np.where((cosmodc2_df_100k['baseDC2/sm'].values>0)&(cosmodc2_df_100k['baseDC2/sfr'].values>0))
prop_cosmodc2 = np.empty((len(cosmodc2_df_100k['baseDC2/sm'].values[w]), 2))
prop_cosmodc2[:, 0] = np.log10(cosmodc2_df_100k['baseDC2/sm'].values[w])
prop_cosmodc2[:, 1] = np.log10(cosmodc2_df_100k['baseDC2/sfr'].values[w])

prop_skysim = np.empty((len(skysim3_df_100k['log_sm']),2))
prop_skysim[:, 0] = skysim3_df_100k['log_sm'].values
prop_skysim[:, 1] = np.log10(skysim3_df_100k['sfr'].values)

plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(prop_cosmodc2, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$\log(M/M_{\odot})$',r'$\log(SFR[M_{\odot}/yr])$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_skysim, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$\log(M/M_{\odot})$',r'$\log(SFR[M_{\odot}/yr])$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### Stellar mass - Star formation rate plane comparison using *_obs values, which refers to the UniverseMachine values used in galsampling. We refer to them as _um in the plot.

In [ ]:
w = np.where((cosmodc2_df_100k['baseDC2/obs_sm'].values>0)&(cosmodc2_df_100k['baseDC2/obs_sfr'].values>0))
prop_cosmodc2 = np.empty((len(cosmodc2_df_100k['baseDC2/obs_sm'].values[w]), 2))
prop_cosmodc2[:, 0] = np.log10(cosmodc2_df_100k['baseDC2/obs_sm'].values[w])
prop_cosmodc2[:, 1] = np.log10(cosmodc2_df_100k['baseDC2/obs_sfr'].values[w])

prop_skysim = np.empty((len(skysim3_df_100k['obs_sm']),2))
prop_skysim[:, 0] = np.log10(skysim3_df_100k['obs_sm'].values)
prop_skysim[:, 1] = np.log10(skysim3_df_100k['obs_sfr'].values)

plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(prop_cosmodc2, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$\log(M/M_{\odot})_{um}$',r'$\log(SFR[M_{\odot}/yr])_{um}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_skysim, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$\log(M/M_{\odot})_{um}$',r'$\log(SFR[M_{\odot}/yr])_{um}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### Stellar mass - Star formation rate comparison where the quantities are computed for skysim by building the SFH of each galaxy and evaluating them at the time of observation corresponding to the galaxy redshift. This plot uses the SFHs computed with diffmah and diffstar, following the example script kindly provided by Andrew Hearin.

In [ ]:
from diffstar.defaults import DEFAULT_N_STEPS, INDX_K, LGT0, FB, T_BIRTH_MIN
from dsps.constants import T_TABLE_MIN
from diffstar.sfh import get_sfh_from_mah_kern
from dsps.cosmology import age_at_z, DEFAULT_COSMOLOGY
from dsps.utils import cumulative_mstar_formed
from jax import vmap
from jax import jit as jjit
from jax import numpy as jnp

First we get the relevant parameters to build the SFH from the skysim catalog

In [ ]:
def get_fit_params(data, mah_keys=None, ms_keys=None, q_keys=None):
    """Read the mock galaxy data table and return the diffmah and diffstar fit params
    Parameters
    ----------
    data : astropy Table of length (n_h, )
        Synthetic galaxy catalog
    mah_keys:
    ms_keys:
    q_keys:
    Returns
    -------
    mah_params : ndarray of shape (n_h, 4)
    ms_u_params : ndarray of shape (n_h, 5)
    q_u_params : ndarray of shape (n_h, 4)
    """
    if q_keys is None:
        q_keys = DIFFSTAR_U_Q_KEYS
    if ms_keys is None:
        ms_keys = DIFFSTAR_U_MS_KEYS
    if mah_keys is None:
        mah_keys = DIFFMAH_KEYS
    mah_params = np.array([data[key] for key in mah_keys]).T
    u_ms_params = np.array([data[key] for key in ms_keys]).T
    u_q_params = np.array([data[key] for key in q_keys]).T
    return mah_params, u_ms_params, u_q_params

DIFFMAH_KEYS = ["diffmah_logmp_fit", "diffmah_mah_logtc", "diffmah_early_index", "diffmah_late_index"]
_ms_keylist = ("lgmcrit", "lgy_at_mcrit", "indx_lo", "indx_hi", "tau_dep")
DIFFSTAR_U_MS_KEYS = ["diffstar_u_" + key for key in _ms_keylist]
DIFFSTAR_U_Q_KEYS = ["diffstar_u_" + key for key in ("qt", "qs", "q_drop", "q_rejuv")]

mah_params, ms_params, q_params = get_fit_params(skysim3_df_100k,mah_keys=DIFFMAH_KEYS,
                                                 ms_keys=DIFFSTAR_U_MS_KEYS,
                                                 q_keys=DIFFSTAR_U_Q_KEYS)

Building the SFH using the kernel in diffstar

In [ ]:
t_table = np.linspace(T_TABLE_MIN, 10**LGT0, DEFAULT_N_STEPS)
sfh_from_mah_kern = get_sfh_from_mah_kern(n_steps=DEFAULT_N_STEPS,
                                          tacc_integration_min=T_BIRTH_MIN,
                                          tobs_loop='vmap', galpop_loop='vmap')
sfh = sfh_from_mah_kern(t_table, mah_params, ms_params, q_params, LGT0, FB)

Compute the values of the SFR at the age that the galaxy has when observed at redshift z

In [ ]:
t_obs = age_at_z(skysim3_df_100k['redshift'].values, *DEFAULT_COSMOLOGY)

In [ ]:
def multiInterp2(x, xp, fp):
    i = np.arange(x.size)
    j = np.searchsorted(xp, x) - 1
    d = (x - xp[j]) / (xp[j + 1] - xp[j])
    return (1 - d) * fp[i, j] + fp[i, j + 1] * d

sfr = multiInterp2(t_obs, t_table, sfh)

Compute the cumulative formed stellar mass up to t_obs

In [ ]:
_c = [None, 0]
_calc_smh_table_vmap = jjit(vmap(cumulative_mstar_formed, in_axes=_c))

In [ ]:
args_smh_table = (t_table,sfh)
smh_table = _calc_smh_table_vmap(*args_smh_table)
logsmh_table = jnp.log10(smh_table)

In [ ]:
log_sm = multiInterp2(t_obs, t_table, logsmh_table)

Plot the Stellar Mass - Star formation rate plane

In [ ]:
w = np.where((cosmodc2_df_100k['baseDC2/sm'].values>0)&(cosmodc2_df_100k['baseDC2/sfr'].values>0))
prop_cosmodc2 = np.empty((len(cosmodc2_df_100k['baseDC2/sm'].values[w]), 2))
prop_cosmodc2[:, 0] = np.log10(cosmodc2_df_100k['baseDC2/sm'].values[w])
prop_cosmodc2[:, 1] = np.log10(cosmodc2_df_100k['baseDC2/sfr'].values[w])

prop_skysim = np.empty((len(log_sm),2))
prop_skysim[:, 0] = log_sm
prop_skysim[:, 1] = np.log10(sfr)

plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(prop_cosmodc2, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$\log(M/M_{\odot})$',r'$\log(SFR[M_{\odot}/yr])$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_skysim, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$\log(M/M_{\odot})$',r'$\log(SFR[M_{\odot}/yr])$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### LSST rest-frame magnitudes and colours comparison

In [ ]:
wavebands = ['u', 'g', 'r', 'i', 'z', 'y']
lsst_restframe_mags_cosmodc2 = np.array([cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:rest'.format(waveband)].values for waveband in wavebands])
lsst_restframe_mags_skysim = np.array([skysim3_df_100k['LSST_rest_{}'.format(waveband.capitalize())].values for waveband in wavebands])

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(np.swapaxes(lsst_restframe_mags_cosmodc2,0,1), hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$u_{rest}$',r'$g_{rest}$',r'$r_{rest}$',r'$i_{rest}$',r'$z_{rest}$',r'$y_{rest}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(lsst_restframe_mags_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$u_{rest}$',r'$g_{rest}$',r'$r_{rest}$',r'$i_{rest}$',r'$z_{rest}$',r'$y_{rest}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

In [ ]:
lsst_restframe_colours_cosmodc2 = np.array([cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:rest'.format(wavebands[i])].values - cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:rest'.format(wavebands[i+1])].values for i in range(len(wavebands)-1)])
lsst_restframe_colours_skysim = np.array([skysim3_df_100k['LSST_rest_{}'.format(wavebands[i].capitalize())].values-skysim3_df_100k['LSST_rest_{}'.format(wavebands[i+1].capitalize())].values for i in range(len(wavebands)-1)])

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(np.swapaxes(lsst_restframe_colours_cosmodc2,0,1), hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$(u-g)_{rest}$',r'$(g-r)_{rest}$',r'$(r-i)_{rest}$',r'$(i-z)_{rest}$',r'$(z-y)_{rest}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(lsst_restframe_colours_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$(u-g)_{rest}$',r'$(g-r)_{rest}$',r'$(r-i)_{rest}$',r'$(i-z)_{rest}$',r'$(z-y)_{rest}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### LSST observed-frame magnitudes and colours comparison

In [ ]:
wavebands = ['u', 'g', 'r', 'i', 'z', 'y']
lsst_obsframe_mags_cosmodc2 = np.array([cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:observed'.format(waveband)].values for waveband in wavebands])
lsst_obsframe_mags_skysim = np.array([skysim3_df_100k['LSST_obs_{}'.format(waveband)].values for waveband in wavebands])

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(np.swapaxes(lsst_obsframe_mags_cosmodc2,0,1), hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$u_{obs}$',r'$g_{obs}$',r'$r_{obs}$',r'$i_{obs}$',r'$z_{obs}$',r'$y_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(lsst_obsframe_mags_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$u_{obs}$',r'$g_{obs}$',r'$r_{obs}$',r'$i_{obs}$',r'$z_{obs}$',r'$y_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

In [ ]:
lsst_obsframe_colours_cosmodc2 = np.array([cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:observed'.format(wavebands[i])].values - cosmodc2_df_100k['LSST_filters/magnitude:LSST_{}:observed'.format(wavebands[i+1])].values for i in range(len(wavebands)-1)])
lsst_obsframe_colours_skysim = np.array([skysim3_df_100k['LSST_obs_{}'.format(wavebands[i])].values-skysim3_df_100k['LSST_obs_{}'.format(wavebands[i+1])].values for i in range(len(wavebands)-1)])

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(np.swapaxes(lsst_obsframe_colours_cosmodc2,0,1), hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$(u-g)_{obs}$',r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(lsst_obsframe_colours_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$(u-g)_{obs}$',r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### Observed color-redshift relation comparison

In [ ]:
colours = [r'$(u-g)_{obs}$',r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$']
for i in range(len(colours)):
    plt.clf()
    plt.rcParams['figure.figsize']=(10,8)
    plt.rcParams['xtick.labelsize']=25
    plt.rcParams['ytick.labelsize']=25
    plt.scatter(cosmodc2_df_100k['baseDC2/redshift'], lsst_obsframe_colours_cosmodc2[i,:], s=1, color='blue', label='cosmoDC2 v1.1.4')
    plt.scatter(skysim3_df_100k['redshift'], lsst_obsframe_colours_skysim[i,:], s=1, color='red', label='skysim v3.1.0')
    plt.legend(loc='upper right', fontsize=20)
    plt.xlim(0,3.5)
    plt.ylim(-1.5,4.5)
    plt.ylabel(colours[i], fontsize=30)
    plt.xlabel(r'redshift', fontsize=20)
    plt.tight_layout()
    plt.show()

## Load COSMOS2020 catalog

In [ ]:
output_directory = os.getcwd()

if not os.path.isfile(os.path.join(output_directory,'COSMOS2020_CLASSIC_R1_v2.0.fits')):
    os.system('curl -O https://portal.nersc.gov/cfs/lsst/PZ/COSMOS2020_CLASSIC_R1_v2.0.fits '
              '--output-dir {}'.format(output_directory))

cosmos2020 = Table.read(os.path.join(output_directory,'COSMOS2020_CLASSIC_R1_v2.0.fits'), format='fits')

In [ ]:
mask_bad_values = np.where((cosmos2020['CFHT_u_MAG_AUTO'] > 0) & 
                             (cosmos2020['HSC_i_MAG_AUTO'] > 0)&
                             (cosmos2020['HSC_g_MAG_AUTO'] > 0)&
                             (cosmos2020['HSC_r_MAG_AUTO'] > 0) & 
                             (cosmos2020['HSC_z_MAG_AUTO'] > 0)&
                             (cosmos2020['HSC_y_MAG_AUTO'] > 0)&
                             (cosmos2020['UVISTA_J_MAG_AUTO'] > 0) &
                             (cosmos2020['UVISTA_H_MAG_AUTO'] > 0) & 
                             (cosmos2020['UVISTA_Ks_MAG_AUTO'] > 0))

In [ ]:
cosmos2020 = cosmos2020[mask_bad_values]

## Since skysim and cosmoDC2 catalogs contain model galaxies with noiseless properties, we compare them against COSMOS2020 observed galaxies by applying the LSST Y1 gold sample cut of 24.1 in the i-band (see DESC SRM v1.0.2, page 48). We use the HSC i-band in COSMOS since the LSST one is not available.

In [ ]:
skysim3_df_lsst_y1_all = skysim3_df.loc[skysim3_df['LSST_obs_i']<=24.1]
cosmodc2_df_lsst_y1_all = cosmodc2_df.loc[cosmodc2_df['LSST_filters/magnitude:LSST_i:observed']<=24.1]

In [ ]:
mask_cosmos_lsst_y1 = np.where(cosmos2020['HSC_i_MAG_AUTO']<=24.1)
cosmos2020_lsst_y1_all = cosmos2020[mask_cosmos_lsst_y1]
rand_idx = np.random.randint(low=0, high=len(cosmos2020_lsst_y1_all),size=100000)
cosmos2020_lsst_y1 = cosmos2020_lsst_y1_all[rand_idx]

In [ ]:
skysim3_df_lsst_y1 = skysim3_df_lsst_y1_all.sample(n=100000)
cosmodc2_df_lsst_y1 = cosmodc2_df_lsst_y1_all.sample(n=100000)

## Compare physical properties distribution among skysim v3.1.0, cosmoDC2 v1.1.4 and COSMOS2020, and photometric properties distribution between Skysim and COSMOS2020

#### Redshift distribution comparison between cosmoDC2 v1.1.4 and skysim v3.1.0 input model redshifts and COSMOS2020 observed redshifts obtained via Eazy photo-z (maximum a-posteriori).

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25
w = np.where((cosmos2020_lsst_y1['ez_z_phot']>0)&(cosmos2020_lsst_y1['ez_z_phot']<20))
plt.hist(cosmos2020_lsst_y1['ez_z_phot'][w], bins=50, lw=3, ls='solid', color='black', histtype='step',density=True, label='COSMOS2020 (Eazy)')
plt.hist(cosmodc2_df_lsst_y1['baseDC2/redshift'], bins=50, lw=2, ls='solid', color='blue', histtype='step',density=True, label='cosmoDC2 v1.1.4')
plt.hist(skysim3_df_lsst_y1['redshift'], bins=50, lw=2, ls='solid', color='red', histtype='step',density=True, label='skysim v3.1.0')
plt.vlines(np.nanmedian(cosmos2020_lsst_y1['ez_z_phot'][w]),0,1.2, lw=2, ls='dashed', color='black', label=r'$Me(\bar{z}_{COSMOS2020(Eazy)})=$'+'{:.3f}'.format(np.nanmedian(cosmos2020_lsst_y1['ez_z_phot'][w])))
plt.vlines(np.nanmedian(cosmodc2_df_lsst_y1['baseDC2/redshift']),0,1.2, lw=2, ls='dashed', color='blue', label=r'$Me(\bar{z}_{cosmoDC2})=$'+'{:.3f}'.format(np.nanmedian(cosmodc2_df_lsst_y1['baseDC2/redshift'])))
plt.vlines(np.nanmedian(skysim3_df_lsst_y1['redshift']),0,1.2, lw=2, ls='dashed', color='red', label=r'$Me(\bar{z}_{skysim})=$'+'{:.3f}'.format(np.nanmedian(skysim3_df_lsst_y1['redshift'])))
plt.legend(loc='upper right', fontsize=10)
plt.xlim(0,3.5)
plt.xlabel(r'redshift', fontsize=20)
plt.ylabel(r'', fontsize=20)
plt.show()

#### Redshift distribution comparison between cosmoDC2 v1.1.4 and skysim v3.1.0 input model redshifts and COSMOS2020 observed redshifts obtained via LePhare photo-z (median of the likelihood distribution). 

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25
w = np.where((cosmos2020_lsst_y1['lp_zPDF']>0)&(cosmos2020_lsst_y1['lp_zPDF']<20))
plt.hist(cosmos2020_lsst_y1['lp_zPDF'][w], bins=50, lw=3, ls='solid', color='black', histtype='step',density=True, label='COSMOS2020 (LePhare)')
plt.hist(cosmodc2_df_lsst_y1['baseDC2/redshift'], bins=50, lw=2, ls='solid', color='blue', histtype='step',density=True, label='cosmoDC2 v1.1.4')
plt.hist(skysim3_df_lsst_y1['redshift'], bins=50, lw=2, ls='solid', color='red', histtype='step',density=True, label='skysim v3.1.0')
plt.vlines(np.nanmedian(cosmos2020_lsst_y1['lp_zPDF'][w]),0,1.2, lw=2, ls='dashed', color='black', label=r'$Me(\bar{z}_{COSMOS2020(LePhare)})=$'+'{:.3f}'.format(np.nanmedian(cosmos2020_lsst_y1['lp_zPDF'][w])))
plt.vlines(np.nanmedian(cosmodc2_df_lsst_y1['baseDC2/redshift']),0,1.2, lw=2, ls='dashed', color='blue', label=r'$Me(\bar{z}_{cosmoDC2})=$'+'{:.3f}'.format(np.nanmedian(cosmodc2_df_lsst_y1['baseDC2/redshift'])))
plt.vlines(np.nanmedian(skysim3_df_lsst_y1['redshift']),0,1.2, lw=2, ls='dashed', color='red', label=r'$Me(\bar{z}_{skysim})=$'+'{:.3f}'.format(np.nanmedian(skysim3_df_lsst_y1['redshift'])))
plt.legend(loc='upper right', fontsize=10)
plt.xlim(0,3.5)
plt.xlabel(r'redshift', fontsize=20)
plt.ylabel(r'', fontsize=20)
plt.show()

#### Stellar mass - Star formation rate plane comparison with Eazy derived physical properties

In [ ]:
mah_params, ms_params, q_params = get_fit_params(skysim3_df_lsst_y1,mah_keys=DIFFMAH_KEYS,
                                                 ms_keys=DIFFSTAR_U_MS_KEYS,
                                                 q_keys=DIFFSTAR_U_Q_KEYS)
t_table = np.linspace(T_TABLE_MIN, 10**LGT0, DEFAULT_N_STEPS)
sfh_from_mah_kern = get_sfh_from_mah_kern(n_steps=DEFAULT_N_STEPS,
                                          tacc_integration_min=T_BIRTH_MIN,
                                          tobs_loop='vmap', galpop_loop='vmap')
sfh = sfh_from_mah_kern(t_table, mah_params, ms_params, q_params, LGT0, FB)
t_obs = age_at_z(skysim3_df_lsst_y1['redshift'].values, *DEFAULT_COSMOLOGY)
sfr_lsst_y1 = multiInterp2(t_obs, t_table, sfh)
_c = [None, 0]
_calc_smh_table_vmap = jjit(vmap(cumulative_mstar_formed, in_axes=_c))
args_smh_table = (t_table,sfh)
smh_table = _calc_smh_table_vmap(*args_smh_table)
logsmh_table = jnp.log10(smh_table)
log_sm_lsst_y1 = multiInterp2(t_obs, t_table, logsmh_table)

In [ ]:
w = np.where((cosmos2020_lsst_y1['ez_mass']>0)&(10**(cosmos2020_lsst_y1['ez_sfr'])>0))
#idxs_cosmos = np.random.randint(0,len(cosmos2020[w]),100000) # 100k random samples
prop_cosmos = np.empty((len(cosmos2020_lsst_y1['ez_mass'][w]), 2))
prop_cosmos[:, 0] = cosmos2020_lsst_y1['ez_mass'][w]
prop_cosmos[:, 1] = cosmos2020_lsst_y1['ez_sfr'][w]

w = np.where((cosmodc2_df_lsst_y1['baseDC2/sm'].values>0)&(cosmodc2_df_lsst_y1['baseDC2/sfr'].values>0))
prop_cosmodc2 = np.empty((len(cosmodc2_df_lsst_y1['baseDC2/sm'].values[w]), 2))
prop_cosmodc2[:, 0] = np.log10(cosmodc2_df_lsst_y1['baseDC2/sm'].values[w])
prop_cosmodc2[:, 1] = np.log10(cosmodc2_df_lsst_y1['baseDC2/sfr'].values[w])

w = np.where((log_sm_lsst_y1>0)&(sfr_lsst_y1>0))
prop_skysim = np.empty((len(log_sm_lsst_y1[w]),2))
prop_skysim[:, 0] = log_sm_lsst_y1[w]
prop_skysim[:, 1] = np.log10(sfr_lsst_y1[w])

plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(prop_cosmos, hist_kwargs=dict(density=True, linewidth=1.5,), #range=extents,
                    color='black', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_cosmodc2, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_skysim, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'black'), 
         plt.Rectangle((0,0),1,1,fc = 'blue'),
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['COSMOS2020 (Eazy)','cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### Stellar mass - Star formation rate plane comparison with LePhare derived physical properties

In [ ]:
w = np.where((cosmos2020_lsst_y1['lp_mass_med']>0)&(cosmos2020_lsst_y1['lp_SFR_med']>-20))
#idxs_cosmos = np.random.randint(0,len(cosmos2020[w]),100000) # 100k random samples
prop_cosmos_lephare = np.empty((len(cosmos2020_lsst_y1['lp_mass_med'][w]), 2))
prop_cosmos_lephare[:, 0] = cosmos2020_lsst_y1['lp_mass_med'][w]
prop_cosmos_lephare[:, 1] = cosmos2020_lsst_y1['lp_SFR_med'][w]

plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(prop_cosmos_lephare, hist_kwargs=dict(density=True, linewidth=1.5,), #range=extents,
                    color='black', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_cosmodc2, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(prop_skysim, fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$\log(M/M_{\odot})_{obs}$',r'$\log(SFR[M_{\odot}/yr])_{obs}$'],
                    label_kwargs={"fontsize":15}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'black'), 
         plt.Rectangle((0,0),1,1,fc = 'blue'),
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['COSMOS2020 (LePhare)','cosmoDC2 v1.1.4', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

#### HSC observed-frame magnitudes and colours comparison between COSMOS2020 and skysim v3.1.0

In [ ]:
wavebands = ['g', 'r', 'i', 'z', 'y']
hsc_obsframe_mags_skysim = np.array([skysim3_df_lsst_y1['HSC_obs_{}'.format(waveband)].values for waveband in wavebands])
hsc_obsframe_colours_skysim = np.array([skysim3_df_lsst_y1['HSC_obs_{}'.format(wavebands[i])].values-skysim3_df_lsst_y1['HSC_obs_{}'.format(wavebands[i+1])].values for i in range(len(wavebands)-1)])
hsc_obsframe_mags_cosmos = np.array([cosmos2020_lsst_y1['HSC_{}_MAG_AUTO'.format(waveband)] for waveband in wavebands])
hsc_obsframe_colours_cosmos = np.array([cosmos2020_lsst_y1['HSC_{}_MAG_AUTO'.format(wavebands[i])]-cosmos2020_lsst_y1['HSC_{}_MAG_AUTO'.format(wavebands[i+1])] for i in range(len(wavebands)-1)])

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = corner.corner(np.swapaxes(hsc_obsframe_mags_cosmos,0,1), hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='blue', labels=[r'$g_{obs}$',r'$r_{obs}$',r'$i_{obs}$',r'$z_{obs}$',r'$y_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(hsc_obsframe_mags_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), #range=extents,
                    color='red', labels=[r'$g_{obs}$',r'$r_{obs}$',r'$i_{obs}$',r'$z_{obs}$',r'$y_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['COSMOS2020', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

extents=[(-1.5,2.5),(-1.5,2.5),(-1.5,2.5),(-1.5,2.5)]

fig = corner.corner(np.swapaxes(hsc_obsframe_colours_cosmos,0,1), hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='blue', labels=[r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(np.swapaxes(hsc_obsframe_colours_skysim,0,1), fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='red', labels=[r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$'],
                    label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]
axes[1].legend(proxy, ['COSMOS2020', 'skysim v3.1.0'],loc='upper right',fontsize=10)
plt.show()

In [ ]:
colours = [r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$']
for i in range(len(colours)):
    plt.clf()
    plt.rcParams['figure.figsize']=(10,8)
    plt.rcParams['xtick.labelsize']=25
    plt.rcParams['ytick.labelsize']=25
    plt.hist(hsc_obsframe_colours_cosmos[i,:], bins='auto', lw=2, ls='solid', color='blue', histtype='step',density=True, label='COSMOS2020')
    plt.hist(hsc_obsframe_colours_skysim[i,:], bins='auto', lw=2, ls='dashed', color='red', histtype='step',density=True, label='skysim v3.1.0')
    plt.legend(loc='upper right', fontsize=20)
    plt.xlim(-1.5,4.5)
    plt.xlabel(colours[i], fontsize=30)
    plt.ylabel(r'', fontsize=20)
    plt.tight_layout()
    plt.show()

#### Observed color-redshift relation comparison between COSMOS2020 and skysim v3.1.0

In [ ]:
colours = [r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$']
for i in range(len(colours)):
    plt.clf()
    plt.rcParams['figure.figsize']=(10,8)
    plt.rcParams['xtick.labelsize']=25
    plt.rcParams['ytick.labelsize']=25
    plt.scatter(cosmos2020_lsst_y1['ez_z_phot'], hsc_obsframe_colours_cosmos[i,:], s=1, color='blue', label='COSMOS2020 (Eazy)')
    plt.scatter(skysim3_df_lsst_y1['redshift'], hsc_obsframe_colours_skysim[i,:], s=1, color='red', label='skysim v3.1.0',alpha=0.5)
    plt.legend(loc='upper right', fontsize=20)
    plt.xlim(0,3.5)
    plt.ylim(-1.5,4.5)
    plt.ylabel(colours[i], fontsize=30)
    plt.xlabel(r'redshift', fontsize=20)
    plt.tight_layout()
    plt.show()

In [ ]:
colours = [r'$(g-r)_{obs}$',r'$(r-i)_{obs}$',r'$(i-z)_{obs}$',r'$(z-y)_{obs}$']
for i in range(len(colours)):
    plt.clf()
    plt.rcParams['figure.figsize']=(10,8)
    plt.rcParams['xtick.labelsize']=25
    plt.rcParams['ytick.labelsize']=25
    plt.scatter(cosmos2020_lsst_y1['lp_zPDF'], hsc_obsframe_colours_cosmos[i,:], s=1, color='blue', label='COSMOS2020 (LePhare)')
    plt.scatter(skysim3_df_lsst_y1['redshift'], hsc_obsframe_colours_skysim[i,:], s=1, color='red', label='skysim v3.1.0')
    plt.legend(loc='upper right', fontsize=20)
    plt.xlim(0,3.5)
    plt.ylim(-1.5,4.5)
    plt.ylabel(colours[i], fontsize=30)
    plt.xlabel(r'redshift', fontsize=20)
    plt.tight_layout()
    plt.show()

## Load an hdf5 table of rest-frame spectra generated with rail_fsps and rail_dsps using skysim v3.1.0 input properties

In [ ]:
output_directory = os.getcwd()

if not os.path.isfile(os.path.join(output_directory,'skysim_v3.1.0_100k.csv')):
    os.system('curl -O https://portal.nersc.gov/cfs/lsst/PZ/skysim_v3.1.0_100k.csv '
              '--output-dir {}'.format(output_directory))

if not os.path.isfile(os.path.join(output_directory,'skysim_v3.1.0_sed_library.h5')):
    os.system('curl -O https://portal.nersc.gov/cfs/lsst/PZ/skysim_v3.1.0_sed_library.h5 '
              '--output-dir {}'.format(output_directory))

In [ ]:
skysim_for_seds_df = pd.read_csv(os.path.join(output_directory, 'skysim_v3.1.0_100k.csv'))

In [ ]:
with h5py.File(os.path.join(output_directory, 'skysim_v3.1.0_sed_library.h5'), 'r') as h5table:
    idx = h5table['index'][()]
    fsps_ab_magnitudes = h5table['fsps_ab_magnitudes'][()]
    fsps_rest_wavelength_A = h5table['fsps_rest_wavelength_A'][()]
    fsps_rest_Lsolar_Hz = h5table['fsps_rest_Lsolar_Hz'][()]
    dsps_ab_magnitudes = h5table['dsps_ab_magnitudes'][()]
    dsps_rest_wavelength_A = h5table['dsps_rest_wavelength_A'][()]
    dsps_rest_Lsolar_Hz = h5table['dsps_rest_Lsolar_Hz'][()]
    filters_list = [name.decode('utf8') for name in h5table['filters_list'][()]]

In [ ]:
print(filters_list)

#### Comparison of rest-frame spectra generated with FSPS and DSPS on the same input skysim v3.1.0 galaxy properties

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25

mask_fsps = np.where((fsps_rest_wavelength_A>=3500) & (fsps_rest_wavelength_A<=10000))
mask_dsps = np.where((dsps_rest_wavelength_A>=3500) & (dsps_rest_wavelength_A<=10000))

plt.plot(fsps_rest_wavelength_A[mask_fsps], np.nanmedian(fsps_rest_Lsolar_Hz,axis=0)[mask_fsps], 
         lw=3, color='red', label='FSPS')
plt.plot(dsps_rest_wavelength_A[mask_dsps], np.nanmedian(dsps_rest_Lsolar_Hz,axis=0)[mask_dsps], 
         lw=1, color='green', label='DSPS')

plt.legend(loc='upper right', fontsize=20)
plt.xlabel(r'Wavelength [$\AA$]', fontsize=20)
plt.ylabel(r'luminosity density $L_{\nu} [\mathrm{L_{\odot} Hz^{-1}}]$', fontsize=20)
plt.tight_layout()
plt.xlim(3000,10000)
#plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
frac_diff_dspsvsfsps_wrt_fsps = abs((dsps_rest_Lsolar_Hz-fsps_rest_Lsolar_Hz))/fsps_rest_Lsolar_Hz

fifty_percent = np.percentile(frac_diff_dspsvsfsps_wrt_fsps, 50, axis=0,method="closest_observation") 
fifty_percent_idx = np.where(frac_diff_dspsvsfsps_wrt_fsps == fifty_percent)

sixtyeight_percent = np.percentile(frac_diff_dspsvsfsps_wrt_fsps,68,axis=0,method="closest_observation") 
sixtyeight_percent_idx = np.where(frac_diff_dspsvsfsps_wrt_fsps == sixtyeight_percent)

ninetyfive_percent = np.percentile(frac_diff_dspsvsfsps_wrt_fsps,95,axis=0,method="closest_observation") 
ninetyfive_percent_idx = np.where(frac_diff_dspsvsfsps_wrt_fsps == ninetyfive_percent)

ninetynine_percent = np.percentile(frac_diff_dspsvsfsps_wrt_fsps,99,axis=0,method="closest_observation") 
ninetynine_percent_idx = np.where(frac_diff_dspsvsfsps_wrt_fsps == ninetynine_percent)

In [ ]:
plt.clf()
plt.rcParams['figure.figsize']=(10,8)
plt.rcParams['xtick.labelsize']=25
plt.rcParams['ytick.labelsize']=25

mask_dsps = np.where((dsps_rest_wavelength_A>=3500) & (dsps_rest_wavelength_A<=10000))

plt.fill_between(dsps_rest_wavelength_A[mask_dsps], np.full(len(dsps_rest_wavelength_A[mask_dsps]), 0), 
                 fifty_percent[mask_dsps], 
                 facecolor="darkred",label='50%')
plt.fill_between(dsps_rest_wavelength_A[mask_dsps], np.full(len(dsps_rest_wavelength_A[mask_dsps]), 0), 
                 sixtyeight_percent[mask_dsps], 
                 facecolor="red", alpha=0.7,label='68%')
plt.fill_between(dsps_rest_wavelength_A[mask_dsps], np.full(len(dsps_rest_wavelength_A[mask_dsps]), 0), 
                 ninetyfive_percent[mask_dsps], 
                 facecolor="salmon", alpha=0.5,label='95%')
plt.fill_between(dsps_rest_wavelength_A[mask_dsps], np.full(len(dsps_rest_wavelength_A[mask_dsps]), 0), 
                 ninetynine_percent[mask_dsps], 
                 facecolor="mistyrose", alpha=0.5,label='99%')

emission_lines_names = [r'$\mathrm{OII_{doublet}}$',r'$\mathrm{H_{\beta}}$', r'$\mathrm{OII_1}$+$\mathrm{OIII_2}$', 
                        r'$\mathrm{NII_1}$+$\mathrm{H_{\alpha}}$+$\mathrm{NII_2}$',
                        r'$\mathrm{SII_1}$+$\mathrm{SII_2}$',r'$\mathrm{SIII_{12}}$',r'$\mathrm{SIII_{22}}$']
emission_lines_wavelengths = [3726,4861, 4983, 6562, 6725, 9068, 9531]
colours_emlines = ['purple','blue', 'green', 'orange', 'red', 'magenta', 'pink']
lower_lim_emlines_wave = [3680, 4800, 4950, 6500, 6680, 9018, 9480]
upper_lim_emlines_wave = [3780, 4900, 5050, 6600, 6780, 9108, 9580]
colours_emlines = ['purple','blue', 'green', 'orange', 'red', 'magenta','pink']
y_coord_emlines = [0.07,0.09,0.07,0.09,0.07,0.09,0.07]
for k in range(len(emission_lines_wavelengths)):
    plt.vlines(lower_lim_emlines_wave[k], ymin=0, ymax=0.3, color='black', lw=2)
    plt.vlines(upper_lim_emlines_wave[k], ymin=0, ymax=0.3, color='black', lw=2)
    plt.text(lower_lim_emlines_wave[k]+50, y_coord_emlines[k], emission_lines_names[k], fontsize=20)
    plt.fill_betweenx(np.arange(0,0.31,0.01), lower_lim_emlines_wave[k], upper_lim_emlines_wave[k],
                      alpha=0.3, color=colours_emlines[k])

    
plt.legend(loc='center right', fontsize=20)
plt.xlabel(r'Wavelength [$\AA$]', fontsize=20)
plt.ylabel(r'|DSPS-FSPS|/FSPS', fontsize=20)
plt.tight_layout()
plt.xlim(3000,10000)
plt.yscale('log')
plt.show()

#### Comparison of optical and infrared colours between COSMOS2020 and those obtained via DSPS. We use the same CFHTLS, HSC and VISTA magnitudes as those contained in the COSMOS2020 catalog. We also cut at i=24.1 in the i-band.

In [ ]:
skysim_mags_names = ['cfhtls_u','hsc_g', 'hsc_r', 'hsc_i', 'hsc_z', 'hsc_y','vista_vircam_J', 'vista_vircam_H',
                     'vista_vircam_Ks']
cosmos_mags_names = ['CFHT_u_MAG_AUTO','HSC_g_MAG_AUTO','HSC_r_MAG_AUTO','HSC_i_MAG_AUTO','HSC_z_MAG_AUTO',
                     'HSC_y_MAG_AUTO','UVISTA_J_MAG_AUTO','UVISTA_H_MAG_AUTO','UVISTA_Ks_MAG_AUTO']

In [ ]:
idx_hsc_i = filters_list.index('hsc_i')

dsps_y1_mask = np.where(dsps_ab_magnitudes[:, idx_hsc_i] <= 24.1)

dsps_y1_ab_magnitudes = dsps_ab_magnitudes[dsps_y1_mask]

idx_rand_cosmos2020_y1 = np.random.randint(low=0, high=len(cosmos2020_lsst_y1_all), size=10000)
cosmos2020_lsst_y1_red = cosmos2020_lsst_y1_all[idx_rand_cosmos2020_y1]

dsps_colors = np.empty((len(dsps_y1_ab_magnitudes), 8))
cosmos2020_colors = np.empty((len(cosmos2020_lsst_y1_red), 8))

In [ ]:
for i in range(len(skysim_mags_names)-1):
    idx_mag1 = filters_list.index(skysim_mags_names[i])
    idx_mag2 = filters_list.index(skysim_mags_names[i+1])
    dsps_colors[:, i] = dsps_y1_ab_magnitudes[:, idx_mag1] - dsps_y1_ab_magnitudes[:, idx_mag2]
    cosmos2020_colors[:,i] = cosmos2020_lsst_y1_red[cosmos_mags_names[i]] - cosmos2020_lsst_y1_red[cosmos_mags_names[i+1]]

In [ ]:
plt.clf()

plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

extents = [(-1.5,4.), (-1.5,4.), (-1.5,4.), (-1.5,4.),(-1.5,4.)]

fig = corner.corner(cosmos2020_colors[:,:5], hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='blue', labels=colours[:5], label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(dsps_colors[:,:5], fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='red', labels=colours[:5], label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
                    
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]

axes[3].legend(proxy, ['COSMOS2020','DSPS from skysim v3.1.0'],loc='upper right',fontsize=20)

plt.show()

In [ ]:
plt.clf()

plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

extents = [(-1.5,4.), (-1.5,4.), (-1.5,4.)]

fig = corner.corner(cosmos2020_colors[:,5:], hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='blue', labels=colours[5:], label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
fig = corner.corner(dsps_colors[:,5:], fig=fig, hist_kwargs=dict(density=True, linewidth=1.5), range=extents,
                    color='red', labels=colours[5:], label_kwargs={"fontsize":30}, use_math_text=True,
                    plot_datapoints=False, levels=(1-np.exp(-0.5),1-np.exp(-2),1-np.exp(-4.5)), show_titles=False,
                    title_fmt='.4f')
                    
axes = np.array(fig.axes)
proxy = [plt.Rectangle((0,0),1,1,fc = 'blue'), 
         plt.Rectangle((0,0),1,1,fc = 'red')]

axes[2].legend(proxy, ['COSMOS2020','DSPS from skysim v3.1.0'],loc='upper right',fontsize=20)
plt.show()